# Homework 4 Solutions

1. Download Daily Bars for FB, AAPL, AMZN, NFLX, GOOGL and QQQ from yahoo finance starting 2016-01-01. Use the Adj Close to compute daily returns.

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

univ = ['META','AAPL','AMZN','NFLX','GOOGL','QQQ']
px = yf.download(univ, start="2016-01-01")['Adj Close']
ret = px / px.shift() - 1

[*********************100%***********************]  6 of 6 completed


2. Now, let's compute the beta of FB, AAPL, AMZN, NFLX, GOOGL using QQQ as our benchmark. You can think of this as the beta these stocks have to their industry (tech). In practice,  we have to use some lookback window to compute the beta. Let's use 252 (1 year, excluding wknds/holidays). So, for each day, the betas should be computed using the most recent 252 data points.

While it's possible to use a loop + statsmodels OLS function here, the computation is simple enough we don't need that complexity and overhead. The formula for beta in a univariate regression (with a constant) is correlation*(vol_y/vol_benchmark). Here I computed this quickly using the pandas rolling function.

In [3]:
corr = ret.rolling(252).corr(ret['QQQ'])
vol = ret.rolling(252).std()
beta = (corr*vol).divide(vol['QQQ'],axis=0)  

3. Using the betas, compute an "alpha" on each day. This is also known as a "residual return".

In [4]:
resid = ret - beta.multiply(ret['QQQ'],0)

4. Compare the volatility of the residual returns to that of the original returns. What do you notice?

The residual returns generally have much lower volatility. This is because we've taken out the "beta component" which drives much of the stock returns.

In [5]:
vol = {}
vol['orig'] = ret.std()*np.sqrt(252)
vol['resid'] = resid.std()*np.sqrt(252)
vol = pd.DataFrame(vol).drop('QQQ')
vol

,orig,resid
AAPL,0.293689,0.161240
AMZN,0.294663,0.178961
FB,0.323751,0.224119
GOOGL,0.260801,0.148505
NFLX,0.394258,0.292946


5. Compare the pairwise correlations of the residual returns to that of the original returns. What do you notice?

The pairwise correlations of the residual returns in general are much lower between stocks. This is because we've taken out one of the major common forces (tides) moving these stocks. Also striking is the large drop in correlations with QQQ. The original returns are 0.6-0.8 correlated with QQQ, but the residual returns are nearly 0 correlated with QQQ. Note, however, that they are not perfectly 0 correlated. This is because in practice we have to estimate the beta using rolling windows rather than the full sample

In [6]:
ret.corr()

,AAPL,AMZN,FB,GOOGL,NFLX,QQQ
AAPL,1.000000,0.597157,0.573683,0.642036,0.449424,0.832867
AMZN,0.597157,1.000000,0.608957,0.651436,0.563991,0.758396
FB,0.573683,0.608957,1.000000,0.679038,0.458043,0.721541
GOOGL,0.642036,0.651436,0.679038,1.000000,0.500437,0.830525
NFLX,0.449424,0.563991,0.458043,0.500437,1.000000,0.595634
QQQ,0.832867,0.758396,0.721541,0.830525,0.595634,1.000000


In [7]:
resid.corr()

,AAPL,AMZN,FB,GOOGL,NFLX,QQQ
AAPL,1.000000,-0.061590,-0.049798,-0.168590,-0.109972,-0.048033
AMZN,-0.061590,1.000000,0.088339,0.005021,0.220227,0.049032
FB,-0.049798,0.088339,1.000000,0.189888,0.051855,0.010215
GOOGL,-0.168590,0.005021,0.189888,1.000000,-0.019504,0.059638
NFLX,-0.109972,0.220227,0.051855,-0.019504,1.000000,0.031492
QQQ,-0.048033,0.049032,0.010215,0.059638,0.031492,1.000000


6. Compute the information ratio for each of these stocks and compare that to the sharpe ratio

The IRs are in general lower than the SRs, suggesting most of the performance of these stocks was driven by their exposure to QQQ. AAPL performed the best looking at IR or SR and FB performed the worst. 

In [8]:
df = {}
df['IR'] = resid.mean() / resid.std()*np.sqrt(252)
df['SR'] = ret.mean() / ret.std()*np.sqrt(252)
df = pd.DataFrame(df).drop('QQQ')
df

,IR,SR
AAPL,0.537986,1.233033
AMZN,0.352762,1.137797
FB,-0.144798,0.787312
GOOGL,0.118156,0.999067
NFLX,0.329492,0.966760
